In [39]:
import numpy as np
import torch
import pandas as pd
import chess
import re
import gc

In [40]:
chess_games_raw = pd.read_csv('chess-games/chess_games.csv', usecols=['AN', 'WhiteElo'])
print(chess_games_raw.shape[0])
chess_games = chess_games_raw[chess_games_raw['WhiteElo'] > 2000]
del chess_games_raw
gc.collect()

chess_games = chess_games[['AN']]
chess_games = chess_games[~chess_games['AN'].str.contains('{')]
chess_games = chess_games[chess_games['AN'].str.len() > 20]
print(chess_games.shape[0])


112029
15303


In [28]:
letterToNumber = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7}
numberToLetter = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h'}

In [12]:
def createPieceRepresentation(board):
    s = str(board)
    s = re.sub(f'[^{type}{type.upper()} \n]', '.', s)
    s = re.sub(f'{type}', -1,s)
    s = re.sub(f'{type.upper()}', 1,s)
    s = re.sub('\.', '0', s)

    board_material = []

    for row in s.split('\n'):
        row = row.split(' ')
        row = [int(x) for x in row]
        board_material.append(row)
    return np.array(board_material)

In [13]:
def boardRepresation(board):
    pieces = ['p', 'r', 'n', 'b', 'q', 'k']
    layer = []
    for piece in pieces:
        layer.append(createPieceRepresentation(board, piece))
    board_rep = np.stack(layer)
    return board_rep

In [14]:
def move_represation(move, board):
    board.push_san(move).uci()
    move = str(board.pop())

    from_output_layer = np.zeros((8,8))
    from_row = 8-int(move[1])
    from_col = letterToNumber[move[0]]
    from_output_layer[from_row, from_col] = 1

    to_output_layer = np.zeros((8,8))
    to_row = 8-int(move[3])
    to_col = letterToNumber[move[2]]
    to_output_layer[to_row, to_col] = 1

    return np.stack([from_output_layer, to_output_layer])

In [15]:
def create_move_list(s):
    return re.sub('\d*\.', '', s).split(' ')[:-1]